<a href="https://colab.research.google.com/github/wooheehee/deeplearning-practice/blob/main/CH6_1_MNIST_%EC%88%AB%EC%9E%90%EB%B6%84%EB%A5%98%EA%B8%B0_Autoencoder%2BSoftmax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train.astype('float32'), x_test.astype('float32')
x_train, x_test = x_train.reshape([-1, 784]), x_test.reshape([-1, 784])
x_train, x_test = x_train / 255., x_test / 255.

11490434/11490434 [==============================] - 0s 0us/step


In [3]:
learning_rate_RMSProp = 0.02
learning_rate_GradientDescent = 0.5
num_epochs = 100         
batch_size = 256
display_step = 1        
input_size = 784         
hidden1_size = 128        
hidden2_size = 64

In [4]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.shuffle(60000).batch(batch_size)

In [5]:
def random_normal_intializer_with_stddev_1():
  return tf.keras.initializers.RandomNormal(mean=0.0, stddev=1.0, seed=None)

In [6]:
class AutoEncoder(tf.keras.Model):
  def __init__(self):
    super(AutoEncoder, self).__init__()
    self.hidden_layer_1 = tf.keras.layers.Dense(hidden1_size,
                                                activation='sigmoid',
                                                kernel_initializer=random_normal_intializer_with_stddev_1(),
                                                bias_initializer=random_normal_intializer_with_stddev_1())
    self.hidden_layer_2 = tf.keras.layers.Dense(hidden2_size,
                                                activation='sigmoid',
                                                kernel_initializer=random_normal_intializer_with_stddev_1(),
                                                bias_initializer=random_normal_intializer_with_stddev_1())
    self.hidden_layer_3 = tf.keras.layers.Dense(hidden1_size,
                                                activation='sigmoid',
                                                kernel_initializer=random_normal_intializer_with_stddev_1(),
                                                bias_initializer=random_normal_intializer_with_stddev_1())
    self.output_layer = tf.keras.layers.Dense(input_size,
                                                activation='sigmoid',
                                                kernel_initializer=random_normal_intializer_with_stddev_1(),
                                                bias_initializer=random_normal_intializer_with_stddev_1())
    
  def call(self, x):
    H1_output = self.hidden_layer_1(x)
    H2_output = self.hidden_layer_2(H1_output)
    H3_output = self.hidden_layer_3(H2_output)
    X_reconstructed = self.output_layer(H3_output)

    return X_reconstructed, H2_output

In [7]:
class SoftmaxClassifier(tf.keras.Model):
  def __init__(self):
    super(SoftmaxClassifier, self).__init__()
    self.softmax_layer = tf.keras.layers.Dense(10,
                                               activation='softmax',
                                               kernel_initializer='zeros',
                                               bias_initializer='zeros')
  
  def call(self, x):
    y_pred = self.softmax_layer(x)

    return y_pred

In [8]:
@tf.function
def pretraining_mse_loss(y_pred, y_true):
  return tf.reduce_mean(tf.pow(y_true - y_pred, 2))

In [9]:
@tf.function
def finetuning_cross_entropy_loss(y_pred_softmax, y):
  return tf.reduce_mean(-tf.reduce_sum(y*tf.math.log(y_pred_softmax), axis=[1]))

In [10]:
pretraining_optimizer = tf.optimizers.RMSprop(learning_rate_RMSProp, epsilon=1e-10)

In [11]:
@tf.function
def pretraining_train_step(autoencoder_model, x):
  y_true = x
  with tf.GradientTape() as tape:
    y_pred, _ = autoencoder_model(x)
    pretraining_loss = pretraining_mse_loss(y_pred, y_true)
  gradients = tape.gradient(pretraining_loss, autoencoder_model.trainable_variables)
  pretraining_optimizer.apply_gradients(zip(gradients, autoencoder_model.trainable_variables))

In [12]:
finetuning_optimizer = tf.optimizers.SGD(learning_rate_GradientDescent)

In [13]:
@tf.function 
def finetuning_train_step(autoencoder_model, softmax_classifier_model, x, y):
  with tf.GradientTape() as tape:
    y_pred, extracted_features = autoencoder_model(x)
    y_pred_softmax = softmax_classifier_model(extracted_features)
    finetuning_loss = finetuning_cross_entropy_loss(y_pred_softmax, y)
  autoencoder_encoding_variables = autoencoder_model.hidden_layer_1.trainable_variables + autoencoder_model.hidden_layer_2.trainable_variables
  gradients = tape.gradient(finetuning_loss, autoencoder_encoding_variables + softmax_classifier_model.trainable_variables)
  finetuning_optimizer.apply_gradients(zip(gradients, autoencoder_encoding_variables + softmax_classifier_model.trainable_variables))

In [14]:
@tf.function
def compute_accuracy(y_pred_softmax, y):
  correct_prediction = tf.equal(tf.argmax(y_pred_softmax, 1), tf.argmax(y, 1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

  return accuracy

In [15]:
AutoEncoder_model = AutoEncoder()

In [16]:
SoftmaxClassifier_model = SoftmaxClassifier()

In [17]:
for epoch in range(num_epochs):
  # 모든 배치들에 대해서 최적화를 수행합니다.
  for batch_x, _ in train_data:
    _, pretraining_loss_print = pretraining_train_step(AutoEncoder_model, batch_x), pretraining_mse_loss(AutoEncoder_model(batch_x)[0], batch_x)
  # 지정된 epoch마다 학습결과를 출력합니다.
  if epoch % display_step == 0:
    print("반복(Epoch): %d, Pre-Training 손실 함수(pretraining_loss): %f" % ((epoch + 1), pretraining_loss_print))
print("Step 1 : MNIST 데이터 재구축을 위한 오토인코더 최적화 완료(Pre-Training)")

반복(Epoch): 1, Pre-Training 손실 함수(pretraining_loss): 0.051933
반복(Epoch): 2, Pre-Training 손실 함수(pretraining_loss): 0.044133
반복(Epoch): 3, Pre-Training 손실 함수(pretraining_loss): 0.037276
반복(Epoch): 4, Pre-Training 손실 함수(pretraining_loss): 0.036266
반복(Epoch): 5, Pre-Training 손실 함수(pretraining_loss): 0.034023
반복(Epoch): 6, Pre-Training 손실 함수(pretraining_loss): 0.037164
반복(Epoch): 7, Pre-Training 손실 함수(pretraining_loss): 0.029869
반복(Epoch): 8, Pre-Training 손실 함수(pretraining_loss): 0.030387
반복(Epoch): 9, Pre-Training 손실 함수(pretraining_loss): 0.030131
반복(Epoch): 10, Pre-Training 손실 함수(pretraining_loss): 0.027222
반복(Epoch): 11, Pre-Training 손실 함수(pretraining_loss): 0.028287
반복(Epoch): 12, Pre-Training 손실 함수(pretraining_loss): 0.029076
반복(Epoch): 13, Pre-Training 손실 함수(pretraining_loss): 0.022610
반복(Epoch): 14, Pre-Training 손실 함수(pretraining_loss): 0.024619
반복(Epoch): 15, Pre-Training 손실 함수(pretraining_loss): 0.024744
반복(Epoch): 16, Pre-Training 손실 함수(pretraining_loss): 0.026640
반복(Epoch): 17, Pr

In [18]:
for epoch in range(num_epochs + 100):
  for batch_x, batch_y in train_data:
    batch_y = tf.one_hot(batch_y, depth=10)
    _, finetuning_loss_print = finetuning_train_step(AutoEncoder_model, SoftmaxClassifier_model, batch_x, batch_y), finetuning_cross_entropy_loss(SoftmaxClassifier_model(AutoEncoder_model(batch_x)[1]), batch_y)
  if epoch % display_step == 0:
    print("반복(Epoch): %d, Fine-tuning 손실 함수(finetuning_loss): %f" % ((epoch + 1), finetuning_loss_print))
print("Step 2 : MNIST 데이터 분류를 위한 오토인코더+Softmax 분류기 최적화 완료(Fine-Tuning)")

반복(Epoch): 129, Fine-tuning 손실 함수(finetuning_loss): 0.021641
반복(Epoch): 130, Fine-tuning 손실 함수(finetuning_loss): 0.075768
반복(Epoch): 131, Fine-tuning 손실 함수(finetuning_loss): 0.017730
반복(Epoch): 132, Fine-tuning 손실 함수(finetuning_loss): 0.022574
반복(Epoch): 133, Fine-tuning 손실 함수(finetuning_loss): 0.011989
반복(Epoch): 134, Fine-tuning 손실 함수(finetuning_loss): 0.020820
반복(Epoch): 135, Fine-tuning 손실 함수(finetuning_loss): 0.025474
반복(Epoch): 136, Fine-tuning 손실 함수(finetuning_loss): 0.056525
반복(Epoch): 137, Fine-tuning 손실 함수(finetuning_loss): 0.022043
반복(Epoch): 138, Fine-tuning 손실 함수(finetuning_loss): 0.006959
반복(Epoch): 139, Fine-tuning 손실 함수(finetuning_loss): 0.011196
반복(Epoch): 140, Fine-tuning 손실 함수(finetuning_loss): 0.018533
반복(Epoch): 141, Fine-tuning 손실 함수(finetuning_loss): 0.032099
반복(Epoch): 142, Fine-tuning 손실 함수(finetuning_loss): 0.022752
반복(Epoch): 143, Fine-tuning 손실 함수(finetuning_loss): 0.039507
반복(Epoch): 144, Fine-tuning 손실 함수(finetuning_loss): 0.016629
반복(Epoch): 145, Fine-tun

In [19]:
print("정확도(오토인코더+Softmax 분류기): %f" % compute_accuracy(SoftmaxClassifier_model(AutoEncoder_model(x_test)[1]), tf.one_hot(y_test, depth=10)))

정확도(오토인코더+Softmax 분류기): 0.961200
